# Retail Store 상품 분류 LoRA 파인 튜닝

---

> 이 노트북은 SageMaker Studio Notebook PyTorch 1.13 Python 3.9 docker image 와 ml.t3.medium 에서 테스트 되었습니다. :


# 0. 선수 사항
- 아래 노트북의 데이타 준비 노트북을 실행을 먼저 해야 합니다.
    - Kor-LLM-On-SageMaker/3-Lab03-Fine-Tuning/1.Instruction-Fine-Tune/retail_store_data/1_create_retail_dataset.ipynb

# 1. 기본 환경 설정

In [5]:
# install_needed = True
install_needed = False

import sys

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip # --quiet
    !{sys.executable} -m pip install -U sagemaker # --quiet    
    !{sys.executable} -m pip install transformers==4.26.0 # --quiet
    !{sys.executable} -m pip install datasets[s3]==2.9.0 # --quiet
    !{sys.executable} -m pip install py7zr # --quiet  
    !{sys.executable} -m pip install langchain==0.0.279 # --quiet       


In [6]:
# ! grep list | grep sagemaker
# ! grep list | grep transformers
# ! grep list | grep datasets
# ! grep list | grep langchain

In [7]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('../../common_code')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it.



In [8]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)


print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")
print(f"bucket: {sagemaker_session_bucket}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::057716757052:role/gen_ai_gsmoon
sagemaker bucket: sagemaker-us-east-1-057716757052
sagemaker session region: us-east-1
bucket: sagemaker-us-east-1-057716757052


# 2. LLM 모델 지정

In [9]:
model_id = "nlpai-lab/kullm-polyglot-12.8b-v2"
model_name = model_id.split('/')[1]
model_name = model_name.split('.')[0]
print("model_name: ", model_name)

model_name:  kullm-polyglot-12


# 3. 데이터 세트 로딩


In [10]:
from scripts.RetailStoreData import RetailStoreData

dataset_name = 'ko-retail-store-data'
download_folder = "retail_store_data"

# KoAlpacaDataset = KoAlpacaData(download_folder)
RetailStoreDataset = RetailStoreData(data_file_name="retail_demo_train.json",
                                     download_folder=download_folder, 
                                     split_rate = None, 
                                     is_split= False, 
                                     is_download=False)

Existing data is used
data_file_path:  retail_store_data/retail_demo_train.json


In [11]:
train_dataset = RetailStoreDataset.train_dataset
len(train_dataset)


1496

## (샘플) 데이터 세트 로딩
- 일부만을 로직 체크를 위해서 sample_size 지정해서 할 수 있습니다.

In [12]:
def sample_dataset(DatasetObject, dataset, sample_size, sample_json_file):
    '''
    주어진 dataset 에서 sample_size 만큼 추출 함.
    '''
    sample_json_file = f"{sample_json_file}"
    sample_dataset = DatasetObject.sample_dataset(dataset, sample_size, sample_json_file)
    return sample_dataset


In [13]:

sample_size = len(train_dataset)
# sample_size = 100    
train_sample_json_file = f"{download_folder}/ko_retail_store_data_train_sample.json"
sample_train_dataset = sample_dataset(
                                      DatasetObject = RetailStoreDataset,
                                      dataset = train_dataset,
                                      sample_size = sample_size,
                                      sample_json_file = train_sample_json_file)

Size of sample dataset:  293522


Using custom data configuration default-37baa400131cd882


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-37baa400131cd882/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


# 4. 훈련 데이터 셋 준비

In [14]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.model_max_length = 2048 # overwrite wrong value

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### 데이터 세트 템플릿 정의

In [15]:
import os
os.system("mkdir -p templates")

0

In [16]:
list(sample_train_dataset.features)

['description_ko', 'single_category_ko']

In [17]:
import json

prompt_template = f"다음의 상품 설명을 분류하세요.:\n{{product_description}}\n---\n답변:\n{{category}}{{eos_token}}"
prompt_template

'다음의 상품 설명을 분류하세요.:\n{product_description}\n---\n답변:\n{category}{eos_token}'

### 데이터 세트를 템플릿 형태로 생성

In [18]:
from random import randint
from itertools import chain
from functools import partial


# custom instruct prompt start
# prompt_template = f"Summarize the chat dialogue:\n{{dialogue}}\n---\nSummary:\n{{summary}}{{eos_token}}"

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = prompt_template.format(product_description=sample["description_ko"],
                                            category=sample["single_category_ko"],
                                            eos_token=tokenizer.eos_token)
    return sample

prompt_dataset = sample_train_dataset.map(template_dataset, 
                                          remove_columns=list(sample_train_dataset.features))
# prompt_dataset = prompt_dataset['train']

  0%|          | 0/1496 [00:00<?, ?ex/s]

In [19]:
print(prompt_dataset[randint(0, len(prompt_dataset))]["text"])

다음의 상품 설명을 분류하세요.:
이 여성용 게인즈버러 재킷은 도시에 딱 맞습니다.
---
답변:
의복|재킷<|endoftext|>


### 데이터 세트를 Transformer 입력 형태로 인코딩

In [20]:
# apply prompt template per sample

# empty list to save remainder from batches to use in next batch
remainder = {"input_ids": [], "attention_mask": []}


def chunk(sample, chunk_length=2048):
    # define global remainder variable to save remainder from batches to use in next batch
    global remainder
    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    
    # get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # Split by chunks of max_len.
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result


In [21]:
# tokenize and chunk dataset
lm_dataset = prompt_dataset.map(
    lambda sample: tokenizer(sample["text"], return_token_type_ids=False), batched=True, remove_columns=list(prompt_dataset.features)
).map(
    partial(chunk, chunk_length=1536),
    batched=True,
)
lm_dataset

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 34
})

### 인코딩 데이터 세트를 S3 에 업로딩

In [22]:
bucket_prefix = f'LLM/instructon_data/RetailStore/train'
training_input_path = f's3://{sagemaker_session_bucket}/{bucket_prefix}'
print(f"training dataset to: \n {training_input_path}")

training dataset to: 
 s3://sagemaker-us-east-1-057716757052/LLM/instructon_data/RetailStore/train


In [23]:
# save train_dataset to s3

lm_dataset.save_to_disk(training_input_path)

print(f"uploaded data to: {training_input_path}")


Saving the dataset (0/1 shards):   0%|          | 0/34 [00:00<?, ? examples/s]

uploaded data to: s3://sagemaker-us-east-1-057716757052/LLM/instructon_data/RetailStore/train


# 5. LoRA and bnb int-8 파인 튜닝



In [24]:
import time
# define Training Job Name 
job_name = f'{model_name}-{dataset_name}-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
print("training job_name: ", job_name)

from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters ={
  'model_id': model_id,                                # pre-trained model
  'dataset_path': '/opt/ml/input/data/training', # path where sagemaker will save training dataset
  'epochs': 10,                                         # number of training epochs
  'per_device_train_batch_size': 1,                    # batch size for training
  'lr': 2e-4,                                          # learning rate used during training
}

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_clm.py',      # train script
    source_dir           = 'scripts',         # directory which includes all the files needed for training
    instance_type        = 'ml.g5.2xlarge', # instances type used for the training job
#    instance_type        = 'ml.p4d.24xlarge', # instances type used for the training job    
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    hyperparameters      =  hyperparameters
)

training job_name:  kullm-polyglot-12-ko-retail-store-data-2023-09-24-12-23-37
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [25]:
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=False)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: kullm-polyglot-12-ko-retail-store-data--2023-09-24-12-23-37-886


Using provided s3_resource


In [26]:
training_job_name = huggingface_estimator.latest_training_job.job_name
print("training_job_name: \n", training_job_name)

training_job_name: 
 kullm-polyglot-12-ko-retail-store-data--2023-09-24-12-23-37-886


In [ ]:
huggingface_estimator.logs()

2023-09-24 12:23:38 Starting - Starting the training job...
2023-09-24 12:23:54 Starting - Preparing the instances for training......
2023-09-24 12:25:01 Downloading - Downloading input data...
2023-09-24 12:25:19 Training - Downloading the training image.....................
2023-09-24 12:28:55 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-09-24 12:29:25,707 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-09-24 12:29:25,722 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-09-24 12:29:25,732 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-09-24 12:29:25,733 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-09-24 12:29:25,936 sagemaker-training-toolkit INFO     Installing dependen

## 다음 노트북에서 아래 훈련 잡의 이름을 사용합니다.

In [ ]:
retail_store_training_job_name = training_job_name
%store retail_store_training_job_name

# A. Reference
- LLM 모델에 필요한 template 파일을 정의하는 코드
    - https://dacon.io/en/codeshare/8449
```
# GPT, LLama, Alpaca 등 LLM 모델에 필요한 template 파일을 정의하는 코드입니다.
# 아래 코드는 Alpaca-Lora가 인식하는 template의 형식입니다. 아래처럼 진행해서, 마찬가지로 json 파일로 저장해주시면 됩니다.

prompt_template = {
    "description" : "Alpaca-Lora Custom Template",
    "prompt_input" : (
        "Below is an instruction that describes a task, Paired with an input that provides further context.\n"
        "Writhe a response that appropriately completes the request.\n"
        "### Instruction : \n{instruction}\n\n### Input : \n{input}\n\n### Response:\n"
    ),
    "prompt_no_input" : (
        "Below is an instruction that describes a task.\n"
        "Writhe a response that appropriately completes the request.\n"
        "### Instruction : \n{instruction}\n\n### Input : \n{input}\n\n### Response:\n"
    ),
    "response_split" : "### Response:"
}
```